In [1]:
import os
os.chdir('..')
%load_ext autoreload
%autoreload 2

In [2]:
from comm_agents.data.reference_experiments import RefExperimentMass, RefExperimentCharge
from comm_agents.data.optimal_answers import get_alpha_star, get_phi_star
from ipywidgets import interact
import numpy as  np

In [9]:
# experiment parameters
GOLF_HOLE_LOC_M = .1
GOLF_HOLE_LOC_C = .1
TOLERANCE = .1
PARAM_DICT = dict(
    m=[1e-20, 1e-20],
    q=[1e-16, -1e-15],
    m_ref_m=2e-20,
    q_ref=[-1e-17, -1e-17],
    v_ref_m=2,
    m_ref_c=2e-20,
    v_ref_c=0,
    N=10,
    alpha=[0, 0],
    phi=[0, 0],
    dt=.01,
    d=.1,
    is_golf_game=False,
    gravity=True)
M_RANGES = [1e-20, 5e-20]
Q0_RANGE = [1e-16, 2e-16]
Q1_RANGE = [-1e-15, -2e-15]
V_REF_RANGE = [1, 2]

In [14]:
alpha_range_re1 = np.linspace(0, 0.6, 50)
m_range = list(np.linspace(1e-20, 1e-19, 50))
alpha_range_re2 = np.linspace(0.5, 1, 20)
q_range_re2_1 = np.linspace(1e-16, 2e-16, 50)
q_range_re2_2 = np.linspace(-1e-15, -2e-15, 50)
v_ref_range = [1, 2, 5, 50]
N_reange = [10, 100, 1000]

In [15]:
def get_re1_plot(alpha1, alpha2, m1, m2, v_ref, N, alpha_star):
    rem = RefExperimentMass(**PARAM_DICT)
    rem.N = N
    rem.angle = np.array([alpha1*np.pi, alpha2*np.pi])
    rem.m = np.array([m1, m2])
    rem.v_ref = v_ref
    if alpha_star:
        rem.angle, loss = get_alpha_star(rem)
        hio = rem.check_for_hole_in_one(golf_hole_loc=GOLF_HOLE_LOC_M, tolerance=TOLERANCE)
        print(f'Loss: {loss}, Hole in one: {hio}')
    rem.set_initial_state()
    rem.run()
    rem.visualize(golf_hole_loc=GOLF_HOLE_LOC_M, tolerance=TOLERANCE)
    
interact(get_re1_plot, alpha1=alpha_range_re1, alpha2=alpha_range_re1,
         m1=m_range, m2=m_range, N=N_reange, alpha_star=False, v_ref=v_ref_range)

interactive(children=(Dropdown(description='alpha1', options=(0.0, 0.012244897959183673, 0.024489795918367346,…

<function __main__.get_re1_plot(alpha1, alpha2, m1, m2, v_ref, N, alpha_star)>

In [16]:
def get_re2_plot(m1, m2, v_ref, phi1=.5, phi2=.63, q1=4e-12, q2=-3e-12, N=100, 
                 phi_star=False, y_cap=True):
    req = RefExperimentCharge(**PARAM_DICT)
    req.is_golf_game = True
    req.N = N
    req.y_cap = y_cap
    req.angle = np.array([phi1*np.pi, phi2*np.pi])
    req.q = np.array([q1, q2])
    req.m = np.array([m1, m2])
    req.v_ref = v_ref
    if phi_star:
        req.angle, loss = get_phi_star(req)
        hio = req.check_for_hole_in_one(golf_hole_loc=GOLF_HOLE_LOC_C, tolerance=TOLERANCE)
        print(f'Loss: {loss}, Hole in one: {hio}')
    req.set_initial_state()
    req.run()
    req.visualize(golf_hole_loc=GOLF_HOLE_LOC_C)
    print(req.q_ref)
interact(get_re2_plot, phi1=alpha_range_re2, phi2=alpha_range_re2,
        q1=q_range_re2_1, q2=q_range_re2_2,
         m1=m_range, m2=m_range,
         N=N_reange, phi_star=False,
         v_ref=v_ref_range,
         y_cap=True)

interactive(children=(Dropdown(description='m1', options=(1e-20, 1.183673469387755e-20, 1.36734693877551e-20, …

<function __main__.get_re2_plot(m1, m2, v_ref, phi1=0.5, phi2=0.63, q1=4e-12, q2=-3e-12, N=100, phi_star=False, y_cap=True)>

In [7]:
rem = RefExperimentMass(**PARAM_DICT)
# alpha_star = get_alpha_star(rem)
# phi_star = get_phi_star(req)

In [8]:
from comm_agents.data.optimal_answers import get_alpha_star_bounds
x = get_alpha_star_bounds(rem)

ImportError: cannot import name 'get_alpha_star_bounds' from 'comm_agents.data.optimal_answers' (/Users/henrik/Documents/hsu/comm-agents/comm_agents/data/optimal_answers.py)

In [67]:
rem.check_for_hole_in_one()

array([0.46246485, 0.46246485])

In [68]:
x

      fun: 5.656967283496073e-09
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.27840041, 0.27840041])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 171
      nit: 4
     njev: 57
   status: 2
  success: False
        x: array([1.45287617, 1.45287617])

In [106]:
def get_single_mass_loss_value_new(exp, i, golf_hole_loc):
    distances = [np.linalg.norm(np.array([x, y]) - np.array([golf_hole_loc, 0]))
                 for x, y in zip(exp.x_series[:, i], exp.z_series[:, i])]
    value = min(distances)
    return value

In [110]:
get_single_mass_loss_value_new(rem, 1, GOLF_HOLE_LOC_M)

0.1

In [ ]:
[np.linalg.norm(np.array([x, y]) - np.array(0, golf_hole_loc))
                 for x, y in zip(exp.x_series[:, i], exp.z_series[:, i])]